In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
response= requests.get(url,timeout=5)    #http request
soup= BeautifulSoup(response.text,'html.parser')

In [3]:
table = soup.find('table',{'class':'wikitable sortable'}).tbody   #table data is under tbody
print(table)

<tbody><tr>
<th>2018<br/>rank
</th>
<th>City
</th>
<th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[c]</a></sup>
</th>
<th>2018<br/>estimate
</th>
<th>2010<br/>Census
</th>
<th>Change
</th>
<th colspan="2">2016 land area
</th>
<th colspan="2">2016 population density
</th>
<th>Location
</th></tr>
<tr>
<td>1
</td>
<td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York City">New York City</a></i><sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[d]</a></sup>
</td>
<td align="left"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="450" data-file-width="900" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_New_York.svg/23px-Flag_of_New_York.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_New_York.svg/35px-Flag_of_New_York.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_New_York.svg/46px-Fl

In [4]:
rows=table.find_all('tr')  #table we need is under the <tr> header
cols=[i.text.replace('\n','') for i in rows[0].find_all('th')]     #columns names are under the <th> headers in rows[0]
cols[2]=cols[2].replace('[c]','')     
cols[6]=cols[6]+' (sqmi)'
cols[7]=cols[7]+' (km2)'
print(cols)

['2018rank', 'City', 'State', '2018estimate', '2010Census', 'Change', '2016 land area (sqmi)', '2016 population density (km2)', 'Location']


In [5]:
cols[8]='2016 population density (sqmi)'#changing some column names in accordance with the table
cols[7]='2016 land area (km2)'
cols.append('2016 population density (km2)')

In [6]:
data=[]
for i in range(1,len(rows)):     
    temp=rows[i].find_all('td')
    temp2=[temp[0].text,temp[1].text,temp[2].text,temp[3].text,temp[4].text,temp[5].text,temp[6].text,temp[7].text,temp[8].text,temp[9].text]
    data.append(temp2)

In [7]:
for i in range(0,len(data)):     #cleaning the data by removing some unwanted repeated characters
    for j in range(0,len(cols)):
        data[i][j]=data[i][j].replace('\n','')
        data[i][j]=data[i][j].replace('\xa0','')
        data[i][j]=data[i][j].replace(',','') 

In [8]:
print(data)

[['1', 'New York City[d]', 'New York', '8398748', '8175133', '+2.74%', '301.5sqmi', '780.9km2', '28317/sqmi', '10933/km2'], ['2', 'Los Angeles', 'California', '3990456', '3792621', '+5.22%', '468.7sqmi', '1213.9km2', '8484/sqmi', '3276/km2'], ['3', 'Chicago', 'Illinois', '2705994', '2695598', '+0.39%', '227.3sqmi', '588.7km2', '11900/sqmi', '4600/km2'], ['4', 'Houston[3]', 'Texas', '2325502', '2100263', '+10.72%', '637.5sqmi', '1651.1km2', '3613/sqmi', '1395/km2'], ['5', 'Phoenix', 'Arizona', '1660272', '1445632', '+14.85%', '517.6sqmi', '1340.6km2', '3120/sqmi', '1200/km2'], ['6', 'Philadelphia[e]', 'Pennsylvania', '1584138', '1526006', '+3.81%', '134.2sqmi', '347.6km2', '11683/sqmi', '4511/km2'], ['7', 'San Antonio', 'Texas', '1532233', '1327407', '+15.43%', '461.0sqmi', '1194.0km2', '3238/sqmi', '1250/km2'], ['8', 'San Diego', 'California', '1425976', '1307402', '+9.07%', '325.2sqmi', '842.3km2', '4325/sqmi', '1670/km2'], ['9', 'Dallas', 'Texas', '1345047', '1197816', '+12.29%', '34

In [10]:
df=pd.DataFrame(data,columns=cols)

In [11]:
df.set_index('2018rank')
df.head()

,2018rank,City,State,2018estimate,2010Census,Change,2016 land area (sqmi),2016 land area (km2),2016 population density (sqmi),2016 population density (km2)
0,1,New York City[d],New York,8398748,8175133,+2.74%,301.5sqmi,780.9km2,28317/sqmi,10933/km2
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7sqmi,1213.9km2,8484/sqmi,3276/km2
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3sqmi,588.7km2,11900/sqmi,4600/km2
3,4,Houston[3],Texas,2325502,2100263,+10.72%,637.5sqmi,1651.1km2,3613/sqmi,1395/km2
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6sqmi,1340.6km2,3120/sqmi,1200/km2


In [12]:
for i in cols:                    #converting all columns from 'str' to 'float' or 'int' and removing unnecessary characters from columns
    if i==cols[0] or i==cols[3] or i==cols[4]:
        df[i]=pd.to_numeric(df.loc[:,i])
    elif i==cols[6]:
        df[i]=pd.to_numeric(df.loc[:,i].apply(lambda x:x.strip('sqmi')))
    elif i==cols[7]:
        df[i]=pd.to_numeric(df.loc[:,i].apply(lambda x:x.strip('km2')))
    elif i==cols[8]:
        df[i]=pd.to_numeric(df.loc[:,i].apply(lambda x:x.strip('/sqmi')))
    elif i==cols[9]:
        df[i]=pd.to_numeric(df.loc[:,i].apply(lambda x:x.strip('/km2')))   

In [13]:

df.describe()

,2018rank,2018estimate,2010Census,2016 land area (sqmi),2016 land area (km2),2016 population density (sqmi),2016 population density (km2)
count,314.000000,3.140000e+02,3.140000e+02,314.000000,314.000000,314.000000,314.000000
mean,157.500000,3.023532e+05,2.796192e+05,93.474522,214.939809,4189.464968,1326.398089
std,90.788215,5.865428e+05,5.628653e+05,135.784458,357.890163,3308.500106,1296.096520
min,1.000000,1.001540e+05,0.000000e+00,6.400000,0.500000,175.000000,5.000000
25%,79.250000,1.205110e+05,1.100615e+05,33.775000,69.775000,2299.750000,646.250000
50%,157.500000,1.661260e+05,1.525620e+05,55.200000,111.100000,3445.500000,1017.500000
75%,235.750000,2.642332e+05,2.377558e+05,104.425000,175.900000,4766.750000,1523.750000
max,314.000000,8.398748e+06,8.175133e+06,1706.600000,4420.100000,28317.000000,10933.000000


In [14]:
df.head()

,2018rank,City,State,2018estimate,2010Census,Change,2016 land area (sqmi),2016 land area (km2),2016 population density (sqmi),2016 population density (km2)
0,1,New York City[d],New York,8398748,8175133,+2.74%,301.5,780.9,28317,10933
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7,1213.9,8484,3276
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3,588.7,11900,4600
3,4,Houston[3],Texas,2325502,2100263,+10.72%,637.5,1651.1,3613,1395
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6,1340.6,3120,1200


In [26]:
df.to_csv(r'C:\Users\sanil\Desktop\top_cities.csv',encoding='utf-8')   #change path accordingly